In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

In [3]:
img_h = 48 
img_w = 48
batch_size = 64

In [4]:
train_data = r"D:\Moon_Bhai_Project\train"
test_data = r"D:\Moon_Bhai_Project\test"

In [5]:
def data_generator(data, aug = False):
    if aug:
        dataset = ImageDataGenerator(
            rescale = 1./255,
            rotation_range = 20,
            zoom_range = 0.2,
            shear_range = 0.2,
            featurewise_center = False,
            featurewise_std_normalization=False,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            horizontal_flip = True,
            fill_mode = 'nearest')
    else:
        dataset = ImageDataGenerator(
            rescale = 1./255)
    
    return dataset.flow_from_directory(
            data, 
            target_size = (img_h, img_w), 
            color_mode = 'grayscale',
            shuffle = True,
            class_mode = 'categorical',
            batch_size = batch_size)

In [6]:
train_data_gen = data_generator(train_data, True)
validation_data_gen = data_generator(test_data)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv_1"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool_1"))         
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv_2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool_2"))         
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same',name="conv_3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool_3"))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name="conv_4"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool_4"))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc_1'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax',name='fc_softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
epochs = 50
steps_per_epoch = train_data_gen.n // batch_size
validation_steps = validation_data_gen.n // batch_size

history = model.fit(
            x = train_data_gen,
            validation_data = validation_data_gen,
            epochs = epochs,
            steps_per_epoch = steps_per_epoch,
            validation_steps = validation_steps,
            shuffle = True)

model_path = r"D:\Moon_Bhai_Project\model"

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
checkpoint = ModelCheckpoint(model_path+"model_weights.h5", monitor='val_accuracy', verbose=1, save_weights_only=True,
                             save_best_only=True, mode='max')
callbacks_list = [reduce_lr,checkpoint]


Epoch 1/50
448/448 [==============================] - 264s 587ms/step - loss: 2.0162 - accuracy: 0.2445 - val_loss: 1.9750 - val_accuracy: 0.2522
Epoch 2/50
448/448 [==============================] - 78s 173ms/step - loss: 1.7613 - accuracy: 0.2758 - val_loss: 1.9662 - val_accuracy: 0.2453
Epoch 3/50
448/448 [==============================] - 80s 178ms/step - loss: 1.6959 - accuracy: 0.3197 - val_loss: 1.6092 - val_accuracy: 0.3658
Epoch 4/50
448/448 [==============================] - 81s 181ms/step - loss: 1.5971 - accuracy: 0.3703 - val_loss: 1.6112 - val_accuracy: 0.3449
Epoch 5/50
448/448 [==============================] - 82s 183ms/step - loss: 1.5246 - accuracy: 0.4070 - val_loss: 1.5674 - val_accuracy: 0.3732
Epoch 6/50
448/448 [==============================] - 78s 173ms/step - loss: 1.4752 - accuracy: 0.4291 - val_loss: 1.3833 - val_accuracy: 0.4655
Epoch 7/50
448/448 [==============================] - 76s 170ms/step - loss: 1.4407 - accuracy: 0.4427 - val_loss: 1.3406 - val_a

In [9]:
model.save('emotion_detection.h5')